In [ ]:
from bs4 import BeautifulSoup,Tag
import requests 
import re  


header = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36 Edge/16.16299'

}

#  替换字符串
def replace_str(source_str, regex, replace_str = ''):         
     
    str_info = re.compile(regex)         
    return str_info.sub(replace_str, source_str)

# 判断soup的子节点长度是否为1，是则递归调用，直到子节点长度不为1，返回soup
def is_single_child_data(soup):
    
    children = list(soup.children)
    num_children = [child for child in children if child.name is not None]        
    if len(num_children) == 1:                  
        return is_single_child_data(num_children[0])          
    else:          
        return soup  

# 获得soup最长的孩子节点
def max_child_data(soup):
        
    max_child = None 
    max_length = 0 

    # 遍历soup的所有直接子节点     
    for child in soup.children:         
        if isinstance(child, Tag):  
            # 确保它是一个标签             
            text_length = len(child.get_text(strip=True))  # 获取去除空白后的文本长度             
            if text_length > max_length:  
                # 检查这是否是目前找到的最长的文本                 
                max_length = text_length                 
                max_child = child      
    
    return max_child  # 返回文本最长的子节点，如果没有找到则返回 None

# 递归处理soup
def recursive_process(soup):     
    # 检查是否只有一个子节点，如果是则深入     
    soup = is_single_child_data(soup)  

    if is_item(soup):         
        # 如果是我们想要的item，则返回soup         
        return soup
  
    
    # 在当前层找到最长的子节点     
    new_soup = max_child_data(soup)   
       
    return recursive_process(new_soup)

# re正则处理html
def re_rules(response):


    # 正则获得body里的内容
    html = re.search(r'<body.*</body>', response.text, re.S).group(0)
    # 去掉script标签
    html = replace_str(html, '(?i)<script(.|\n)*?</script>') #(?i)忽略大小写   
    # 去掉style标签    
    html = replace_str(html, '(?i)<style(.|\n)*?</style>')
    # 去掉form标签(有些数据放到form里，坑！)
    # html = replace_str(html, '(?i)<form(.|\n)*?</form>') 
    # 去掉option标签
    html = replace_str(html, '(?i)<option(.|\n)*?</option>')
    # 去掉input标签
    html = replace_str(html, '(?i)<input(.|\n)*?>') 
    # 去掉img标签
    html = replace_str(html, '(?i)<img(.|\n)*?>')
    # 去掉注释    
    html = replace_str(html, '<!--(.|\n)*?-->')       
    # 去掉标签属性  
    html = replace_str(html, '(?!&[a-z]+=)&[a-z]+;?', ' ') 

    return html

# 处理soup
def process_soup(soup):
    # 去掉文本内容小于7的标签 # 比如<li><a href="http://www.gdtzb.com/kefu/show/33/" id="top-1">入网指导</a></li>的的入网指导长度小于7，舍去
    for tag in soup.find_all():
        
        # 但是文本里不包括数字
        if len(tag.get_text().strip()) < 7 and not re.search(r'\d', tag.get_text()):
            tag.decompose()

    # 去掉空的标签
    for tag in soup.find_all():

        if tag.get_text().strip() == '':         
            tag.decompose()

    return soup


# 判断是否是所需要的item
def is_item(soup):

      # 如果存在大量相似的子节点，说明是li节点，直接返回soup      # 问题1     
    child_type = list([child.name for child in soup.children if child.name is not None])     # 寻找child_type中出现次数最多的元素     
    max_child_type = max(child_type, key = child_type.count)     
    if child_type.count(max_child_type) > 4:         
        return True


def get_response(url):     
    response = requests.get(url, headers=header)        
    response.encoding = 'utf-8'  # 设置编码格式为utf-8     
    return response


url = "http://www.zbwmy.com/150/151/index.html"

response = get_response(url)  # 获取网页源代码

html = re_rules(response)  # 正则处理html

soup = BeautifulSoup(html, 'html.parser')  

soup = process_soup(soup)

soup = recursive_process(soup)

print(soup.prettify())





from sklearn.cluster import KMeans  
from sklearn.feature_extraction.text import TfidfVectorizer  
import numpy as np 

# # 特征提取：这里我们使用原始html标签
# texts = [tag for tag in soup.find_all()] 

# print(texts)

# 去掉文本内容小于7的标签 # 比如<li><a href="http://www.gdtzb.com/kefu/show/33/" id="top-1">入网指导</a></li>的的入网指导长度小于7，舍去





# vectorizer = TfidfVectorizer(max_features=10) 
# X = vectorizer.fit_transform(texts)  # 应用K-均值聚类 
# kmeans = KMeans(n_clusters=3) 
# kmeans.fit(X)  

# # 输出聚类结果
# for i in range(3):     
#     print('第{}类:'.format(i))     
#     for j in np.where(kmeans.labels_ == i)[0]:         
#         print(texts[j]) 
        







In [ ]:

import requests


url = 'http://www.cqtic.net/cmssite/h/list/cms_xwzx_xwzx.jsp?id=895ea9a8-b5f4-4a22-952f-308008a02255&parentId=597dc378-b858-4076-8cd5-65791b9983a7&m=cms_xwzx_gkztb'

response = requests.get(url, headers=header)

response.encoding = 'utf-8'

html = response.text

print(html)

In [2]:

from bs4 import BeautifulSoup,Tag
import numpy as np
import requests 
import re  
from typing import List
import pymysql
from IPython.display import clear_output
from dateutil import parser
from datetime import datetime,timedelta


header = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36 Edge/16.16299'
}

#  替换字符串
def replace_str(source_str:str, regex:str, replace_str = '')->str:         
     
    str_info = re.compile(regex)         
    return str_info.sub(replace_str, source_str)

# 判断soup的子节点长度是否为1，是则递归调用，直到子节点长度不为1，返回soup
def is_single_child_data(soup:Tag)->List[Tag]:
    
    children = list(soup.children)
    num_children = [child for child in children if child.name is not None] 
         
    if len(num_children) == 1:                  
        return is_single_child_data(num_children[0])          
    else:          
        return num_children  

# 获得soup最长的孩子节点
def max_child_data(children:List[Tag])->Tag:
        
    max_child = None 
    max_length = 0 

    # 遍历soup的所有直接子节点     
    for child in children:         
        if isinstance(child, Tag):  
            # 确保它是一个标签     
            cleaned_text = re.sub(r"\s+", "", child.get_text(strip=True))        
            text_length = len(cleaned_text)  # 获取去除空白后的文本长度   
                 
            if text_length > max_length:  
                # 检查这是否是目前找到的最长的文本                 
                max_length = text_length                 
                max_child = child      
    
    return max_child  # 返回文本最长的子节点，如果没有找到则返回 None


# 递归处理soup
def recursive_process(soup:Tag,oldsoup:Tag)->Tag:     
    # 检查是否只有一个子节点，如果是则深入     
    soup = is_single_child_data(soup)  
    
    num = cal_item_std(soup)    # 计算newsoup标准差
    if oldsoup:
        oldnum = cal_item_std(oldsoup)  # 计算oldsoup标准差
        # 最后一层，返回
        if num == None:
            return oldsoup
        # num 和 oldnum 绝对值差小于1，返回
        if num > oldnum - 3 and oldnum < 50 :    # 如果soup标准差大于oldsoup标准差，返回
            return oldsoup
  

    # 在当前层找到最长的子节点     
    new_soup = max_child_data(soup)   
       
    return recursive_process(new_soup,soup)

# re正则处理html
def re_rules(response:str)->str:

    #如果含有body结构则正则获得body里的内容
    if re.search(r'<body.*</body>', response, re.S):
        html = re.search(r'<body.*</body>', response, re.S).group(0)
    else:
        html = response
    
    # 去掉script标签
    html = replace_str(html, '(?i)<script(.|\n)*?</script>') #(?i)忽略大小写   
    # 去掉style标签    
    html = replace_str(html, '(?i)<style(.|\n)*?</style>')
    # 去掉form标签(有些数据放到form里，坑！)
    # html = replace_str(html, '(?i)<form(.|\n)*?</form>') 
    # 去掉option标签
    html = replace_str(html, '(?i)<option(.|\n)*?</option>')
    # 去掉input标签
    html = replace_str(html, '(?i)<input(.|\n)*?>') 
    # 去掉img标签
    html = replace_str(html, '(?i)<img(.|\n)*?>')
    # 去掉注释    
    html = replace_str(html, '<!--(.|\n)*?-->')       
    # 去掉标签属性  
    html = replace_str(html, '(?!&[a-z]+=)&[a-z]+;?', ' ') 

    return html

# 处理soup
def process_soup(soup:Tag)->Tag:
    # 去掉文本内容小于7的标签 # 比如<li><a href="http://www.gdtzb.com/kefu/show/33/" id="top-1">入网指导</a></li>的的入网指导长度小于7，舍去
    for tag in soup.find_all('a'):
        
            # 但是文本里不包括数字
            if len(tag.get_text().strip()) < 6 and not re.search(r'\d', tag.get_text()):
                tag.decompose()

    # 去掉空的标签
    for tag in soup.find_all():

        if tag.get_text().strip() == '':         
            tag.decompose()



    return soup


# 判断是否是所需要的item
def cal_item_std(children:List[Tag])->float:
    
    if not children:
        return None

    # 检查文本长度相似性     
    text_lengths = [len(re.sub(r"\s+", "", child.get_text(strip=True))) for child in children]

    # 如果长度text_lengths大于5，省去最小的一个值,这个值可能是标题，分页等，干扰我们的判断
    if len(text_lengths) > 5:
        text_lengths.remove(min(text_lengths))

    num = np.std(text_lengths) 
    
    return num



#  获取网页源码 get请求
def get_response(url:str)->str:
    response = requests.get(url, headers=header)   
    response.encoding = 'utf-8'  # 设置编码格式为utf-8
    return response.text


# 从mysql数据库获取url
def urls_from_mysql()->List[str]:

        host = '127.0.0.1'      
        user = 'root'      
        password = '123456'      
        database = 'ceeg'     
   
        connection = pymysql.connect(         host=host,         user=user,         password=password,         database=database     )     
        
        cursor = connection.cursor()

        sql = "select root_url from url_params_test_more where id < 15000"  
        cursor.execute(sql)    
        urls = cursor.fetchall()

        cursor.close()
        connection.close()

        return urls


#  从文本中提取标题
def find_title(text):          
    title=re.findall('<title>(.+)</title>',text)          
    return title

# 添加http
def start_http(baseurl,url):         
    if url.startswith("http"):             
        return url        
    else:   
        if url.startswith("/"):  
            return baseurl + url[1:]   
        else:
            return baseurl + url


#  解析url
def parse_url(tag:Tag)->str:

    # 如果tag是a标签，则直接处理该a标签     
    if tag.name == 'a':         
            a_tag = tag     
    else:  
            # 否则查找tag内部的所有a标签         
            a_tag = tag.find_all('a') if tag.find_all('a') else None 

            if not a_tag:         
                return None,None

            # 获得a标签中文本最长的一个a的href                  
            a_tag = max(a_tag, key = lambda x: len(x.get_text(strip=True)))


    url = a_tag.get('href')
    # 不纯在href属性，返回None
    if not url:
        return None,None
     
    cleaned_link = re.sub(r"\s+", "", url)


    title = a_tag.get_text(strip=True)

    cleaned_title = re.sub(r"\s+", "", title)  # 获得文本最长的title


    return cleaned_link, cleaned_title

# parser.parse
def only_one_date(text:List)->datetime:

    extracted_date = None
    for key in text:         
            try:
                extracted_date = parser.parse(key, fuzzy=False).date()
                # if extracted_date < datetime.now().date():

                break         
            except:
                pass

    return extracted_date


# parser.parse
def many_date(text)->datetime:      
    
    extracted_date = []    
    for key in text:                  
        if(len(key) > 2):             
            try:                 
                extracted_date.append(parser.parse(key, fuzzy=False).date())    

            except:                 
                pass      
       
     # 返回最靠近目前时间的日期         
    now = datetime.now()        
    return min(extracted_date, key=lambda x: abs(x - now))



def remove_chinese_characters(text:List[str])->List[str]:     
    chinese_punctuation_pattern = r'[\u3000-\u303F\uFF01-\uFF0F\uFF1A-\uFF20\uFF3B-\uFF40\uFF5B-\uFF65]'  
   

    new_text = []
    for item in text:         
        # 正则表达式匹配汉字范围[\u4e00-\u9fa5]，使用re.sub函数替换匹配到的汉字为空字符串     
        item = re.sub(r'[\u4e00-\u9fa5]', '', item) 
        item = re.sub(chinese_punctuation_pattern, '', item) 
        item = item.replace('[', '').replace(']', '').replace('...', '')
        # 去掉空
        item = re.sub(r'\s+', '', item)
        # 去掉年份
        if len(item) == 4:
            item = re.sub(r'\d{4}', '', item)
        if len(item) == 3:             
            item = re.sub(r'\d{3}', '', item)
        if len(item) > 2 and item != '':             
            new_text.append(item)
    
    return new_text



# 正则匹配日期   年月日
def re_date(text:str)->datetime:
    
    pattern = r"""((?P<year>\d{2,4})[-/.])((?P<month>\d{1,2})[-/.])(?P<day>\d{1,2})"""
    regex = re.compile(pattern, re.VERBOSE)          
    
    # 匹配多个数据的话，返回最早的数据         
    data_list = []

    for match in regex.finditer(text):                    
        year = match.group('year')             
        month = match.group('month')             
        day = match.group('day')

        if year and month and day:
            data_list.append(datetime.strptime(f"{year}-{month}-{day}", "%Y-%m-%d"))

     # 返回最靠近目前时间的日期         
    now = datetime.now()         
    if data_list:              
        return min(data_list, key=lambda x: abs(x - now)).date()       
    else:             
        return None

# 正则匹配日期   月日,没有年份
def re_dates(text:str)->datetime: 

    pattern = r"""((?P<month>\d{1,2})[-/.])(?P<day>\d{1,2})"""     
    regex = re.compile(pattern, re.VERBOSE)                  
      # 匹配多个数据的话，返回最早的数据              
          
    for match in regex.finditer(text):

        year = match.group('year')                      
        month = match.group('month')                      
        day = match.group('day')          
        if year and month and day:             
            return datetime.strptime(f"{year}-{month}-{day}", "%Y-%m-%d").date()           
        


#  解析日期，title
def parse_date(tag:Tag)->datetime:


    text = tag.get_text()

#    年月日齐全
    date = re_date(text)
#   年月日不齐全，只有月日，没有年份
    if not date:
        text = tag.get_text().split()
        date = only_one_date(text)    


    # # 每个子标签的文本内容提取出来
    # text = [tag.get_text(strip=True) for tag in tag.find_all()] 
    # text = remove_chinese_characters(text)

    # extracted_date = only_one_date(text) 


    # if not extracted_date:
    #     # tag中有多个时间
    #     extracted_date = many_date(text)
  

    return date



# 获得url的base_url
def get_base_url(url:str)->str:
    #有http https

    pattern = re.compile(r'(http://|https://)(.*?)/')

    base_url = pattern.search(url).group(1) + pattern.search(url).group(2)+"/"

    return base_url
    

# search关键词,是否包含关键词，包含则返回True
def keyword(text:str)->bool:
    
    # 关键词列表
    list = ["油变","油浸","10KV","35KV","220KV","落后产能淘汰","主变","中性点","整流变","配电变压器","低压变压器","高压变压器","配变","电力变压器","变压器","厂用变","66KV","开关柜","环网柜","中置柜","高压柜","低压柜","变电站","配电柜","调压变压器","节能变压器","箱变","电气柜数据"]

    # 如果list里的关键词在text里，则返回True
    for i in list:
        if i in text:
            return True
            
    return False
        

# 过滤关键词,是否包含关键词，包含则返回True
def filter_keyword(text:str)->bool:
    
     # 废弃关键词列表     
    list =["中标","成交","候选人","结果","合同","南网","南方电网","国网","国家电网","招标失败","采购失败","环境影响","环评","废标","流标","评标","开标","入围","监理","水土保持","竣工","单一来源","单源直接采购","勘察设计","工程勘察","验收","备案","审计","审批","评审","核准","受理","批复","政府采购意向","地质灾害","灾害危险性评估","可研性","可研初设","可研设计","可行性研究","可行性报告","预防性试验","预防性检测","预防性测试","完成公示","中止公告","终止公告","终止招标","中选公告","失败公告","终止","编制","开标记录","设计服务","技术服务","安全评估","安全评价","安全防护","影响评价","消防","可研报告","维护检修","外委维护","检修维护","运行维护","运维","维保","维修","代维","干式变压器","干式配电变压器","干式电力变压器","控制变压器","变压器处置","挂牌","转让","拍卖","变卖","竞拍","出让","招租","租赁","医疗设备","出租","零星物资","结算","保险","保洁","劳保","劳务分包","劳务外包","劳务招标","外包项目","服务外包","施工专业承包","消缺工","程工程分包","工程专业分包","工程设计","工程咨询","工程造价咨询","工程招标代理","工程采购公示","改造设计","改造施工","施工材料","标识","批前公示","零星材料","评审公示","评价报告","社会稳定风险评估","水土保持报告","影响评估","泰开","成套公司","送变电公司","电力电子公司","物业管理","空调","监控","电梯","路灯专用","灯具","刀具","夹具","金具","工具","锁具","家具","五金","消防物资","配件","附件","硬件","线材","辅材","耗材","管材","资产评估","风险评估","废旧物资处置","无功补偿","补偿装置","补偿设备","预算审核","UPS","GIS","SVG","AIS","PHC","AVC","MPP","KKS","采购与安装","采购及安装","办公用品","异常公告","处置公告","评标报告","在线监测","在线温度监测","监测项目","监测装置","监测系统","非物资","风机塔筒","医用","体检","餐饮","射线","导线","母线","装修","工程类","服务类","迁改工程","送出工程","拆除工程","电缆工程","隔离开关","矿用","赛迪集团","开滦集团","内燃柴电机组","灭火物资","灭火系统","灭火装置","防火材料","防火物资","防火槽盒", "碎石机"]
    
    for i in list:
        if i in text:
            return True
    return False

#  时间范围,是否在时间范围内,在则返回True,不在则返回False
def time_range(date):     
    date_interval = 10     
    current_date = datetime.now().date()     
    target_date = date     
    if target_date < current_date - timedelta(days=date_interval):         
        return False    
    return True 

# 匹配关键字，有效信息返回True
def judge_content(text:str)->bool:
    
    # 是否包含关键词，不包含filter_keyword
    if keyword(text):
        if not filter_keyword(text):
            return True

    return False


def get_date(): 

    from playwright.sync_api import sync_playwright       
    STEALTH_PATH = 'stealth.min.js'         
    with sync_playwright() as p:             
        browser = p.chromium.launch(                  headless=True,                  chromium_sandbox=False,                  ignore_default_args=["--enable-automation"],                  channel="chrome",)         
        ua = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'                   
        content = browser.new_context(user_agent=ua)              
        content.add_init_script(path=STEALTH_PATH)                
        page = content.new_page()                   
        page.goto('https://search.bidcenter.com.cn/search?keywords=%e5%8f%98%e5%8e%8b%e5%99%a8')              
        page.wait_for_timeout(5000)          
        
        return page.content()





def run(url:str):
        
        
        base_url = get_base_url(url)  # 获得url的base_url

        # response = get_response(url)  # 获取网页源码 
        response = get_date() 
        
        # print(response.text)
        html = re_rules(response)  # 正则处理html  
        print(html)
        soup = BeautifulSoup(html, 'html.parser')  # 创建soup对象  
        soup = process_soup(soup) # 处理soup,空白字符，空标签等  
        soup = recursive_process(soup,None) # 递归处理soup，找到最长的子节点，在递归处理，直到找到我们想要的item  
        
        for tag in soup:  
            
            # print(tag.preetify())
            url,title = parse_url(tag)  # 解析url   
            if not url:
                continue
            
            url = start_http(base_url,url)  # 添加http

            date = parse_date(tag)  # 获得日期,和标题   

            # 如果日期为空，爬取详情页抓取时间
            if not date:
                continue
                    # # 解析下一页
                    # response = get_response(url)
                    # html = re_rules(response)
                    # date = re_dates(html)
               
            # # 如果爬取的日期太久，直接结束
            # if not time_range(date): 

            #     break
     
            # if not judge_content(title):
            #     continue


            print(url,title,date)



        # clear_output()  # 清空输出


# urls = urls_from_mysql()     
# for url in urls:         
#     url = url[0]
#     run(url)    


# url = "https://ec.powerchina.cn/zgdjcms//category/bulletinList.html?categoryId=2&page=1"
# url = "https://www.gc-zb.com/search/index.html?keyword=%E5%8F%98%E5%8E%8B%E5%99%A8&h_lx=9&date=90&search_field=0&vague=0&h_province=0&submit=+"
# url = 'http://www.sxylcz.cn/list.php?cla=2&hy=0&addr=0'
# url = "https://www.bidnews.cn/search/"
# url = "http://www.gdtzb.com/zb/search.php?kw=%E5%8F%98%E5%8E%8B%E5%99%A8&areaid=0&type=0"
# url = "https://guangfu.bjx.com.cn/zb/"
# url = "https://www.bidding-crmsc.com.cn/bid"
# url = "https://cg.xidian.edu.cn/hwcggg/index.chtml"
# url = "http://www.haikou.gov.cn/xxgk/szfbjxxgk/cztz/zfcg/cggg/"
# url = "http://www.gzggzy.cn/jyywzfcgzfcgcggg/index.jhtml"
url = "https://www.zbytb.com/zb/search.php?kw=%E5%8F%98%E5%8E%8B%E5%99%A8"
url = "https://search.bidcenter.com.cn/search?keywords=%e5%8f%98%e5%8e%8b%e5%99%a8"
# url = "https://www.chinabidding.cn/cblcn/Extension/index1"
# url = "http://www.casc.com.cn/cas/?cat=13"
# url = "http://www.jtgroup.com.cn/news"
# url = "https://sist.shanghaitech.edu.cn/cggg/list.htm"
# url = "http://58.221.160.226:82/webportal/index/bidnotice/list_t/2.do?cate=2"
# url = "http://www.famens.com/Tradeb-.html"
# url = "http://www.fjgczb.com/articleList.shtml?type=35"
# url = "https://www.longdaoyun.com/search/business-opportunity/type-caigou.html"
url = "https://www.cecbid.org.cn/tender/tender"
url = "https://www.okcis.cn/bn/"
url = "http://shanxi.chinabidding.cc/lists/pid/9/time/7.html"
# url = "http://www.bidizhaobiao.com/advsearch/retrieval_list.do?searchCondition.SearchWord=%E4%B8%BB%E5%8F%98"
url = "http://history.sntba.com/website/news_list.aspx?category_id=53"
run(url)


Error: It looks like you are using Playwright Sync API inside the asyncio loop.
Please use the Async API instead.